In [16]:
from PIL import Image
from music21 import *
import numpy as np
import os

In [2]:
HEIGHT = 300
WIDTH = 10
C = 299792458 # speed of light in m/s
GAMMA = 0.8 #correction for display device

In [31]:
def get_score_name(score_path):
    dot = score_path.find('.')
    slash = score_path.find('/')
    return score_path[:slash] + score_path[slash+1 : dot]

def get_frequencies(score_path):
    s = corpus.parse(score_path)
    return [pitch.frequency for pitch in s.pitches] #flattens chords

def get_color(wavelength, gamma):
    # Formula from http://www.noah.org/wiki/Wavelength_to_RGB_in_Python
    
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
        
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
        
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
        
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
        
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
        
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
        
    else:
        R = 0.0
        G = 0.0
        B = 0.0
        
    R *= 255
    G *= 255
    B *= 255
    
    return (int(R), int(G), int(B))

def scale(frequencies):
    new_fs = []
    visible_max = 790
    visible_min = 405
    
    #To scale logarithmically, we use the equation
    # vis_freq = 405 + a * ln (freq / freq_min)
    # where freq is the frequency being scaled,
    # freq_min the lowest frequency in the list,
    # vis_freq the visible frequency to which we scale,
    # 405 is the minimum visible light frequency,
    # and a is the scalar constant we seek.
    
    freq_max = max(frequencies)
    freq_min = min(frequencies)
    
    # visible_max = 790 = 405 + a * ln(freq_max / freq_min)
    # 385 = a * ln(freq_max / freq_min)
    
    scalar = 385 / np.log(freq_max / freq_min)
    
    for frequency in frequencies:
        new_fs.append(405 + scalar * np.log(frequency / freq_min))
        
    return new_fs


def get_colors(frequencies):
    colors = []
    scaled = scale(frequencies)
    
    for frequency in scaled:
        wavelength = C // (frequency * 1000) #in nanometers
        colors.append(get_color(wavelength, GAMMA))
        
    return colors

def make_image(colors):
    size = (WIDTH * len(colors), HEIGHT)
    image = Image.new('RGB', size)
    
    for index, color in enumerate(colors):
        image.paste(color, (WIDTH*index, 0, WIDTH*(index + 1) - 1, HEIGHT - 1))
    
    return image
    

In [32]:
def visualize_score(score): #takes in filename from music21 corpus, e.g. 'bach/bwv108.6.xml'
    frequencies = get_frequencies(score)
    colors = get_colors(frequencies)
    score_name = get_score_name(score) + '.png'
    directory = os.getcwd()

    make_image(colors).show()
    make_image(colors).save(directory + "\\" + score_name)

In [33]:
visualize_score('chopin/mazurka06-2.krn')